In [1]:
import numpy as np
import pandas as pd
import mmlspark

# help(mmlspark)

In [2]:
spark.conf.set(
  "fs.azure.account.key.amllgbm.blob.core.windows.net",
  "<mykeyhere>")

df = spark.read.csv("wasbs://amllgbm-2018-01-31t15-47-50-037z@amllgbm.blob.core.windows.net/higgs/csv/HIGGS.csv", inferSchema=True, header=False)
dbutils.fs.ls("wasbs://amllgbm-2018-01-31t15-47-50-037z@amllgbm.blob.core.windows.net/higgs/csv/HIGGS.csv")

In [3]:
#data = spark.read.csv('/higgs/csv/HIGGS.csv', inferSchema=True, header=False)
data.show(10)

In [4]:
train, test = data.randomSplit([0.75, 0.25], seed=123)

In [5]:
train.rdd.getNumPartitions()

In [6]:
from timeit import default_timer as timer
start = timer()
from mmlspark import TrainClassifier
from mmlspark import LightGBMClassifier
model = TrainClassifier(model=LightGBMClassifier(numIterations=2, defaultListenPort=12450), labelCol="_c0", numFeatures=256).fit(train)
end = timer()
print(end - start)

In [7]:
from timeit import default_timer as timer
start = timer()
from mmlspark import TrainClassifier
from pyspark.ml.classification import GBTClassifier
gbtmodel = TrainClassifier(model=GBTClassifier(), labelCol="_c0", numFeatures=256).fit(train)
end = timer()
print(end - start)

In [8]:
gbtprediction = gbtmodel.transform(test)

In [9]:
prediction = model.transform(test)

In [10]:
from mmlspark import ComputeModelStatistics
metrics = ComputeModelStatistics(evaluationMetric='AUC', labelCol='_c0', \
                                 scoredLabelsCol='scored_labels', scoresCol='scores').transform(prediction)
metrics.show()

In [11]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
gbtmetric = BinaryClassificationEvaluator(labelCol='_c0', metricName='areaUnderROC', rawPredictionCol="scored_labels").evaluate(gbtprediction)
gbtmetric

In [12]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
metric = BinaryClassificationEvaluator(labelCol='_c0', metricName='areaUnderROC', rawPredictionCol="scores").evaluate(prediction)
metric